<a href="https://colab.research.google.com/github/cwpeng-cn/COLAB_RUNTIME/blob/master/torchreid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/KaiyangZhou/deep-person-reid.git
!pip install -r deep-person-reid/requirements.txt
import os
os.chdir('./deep-person-reid')

Cloning into 'deep-person-reid'...
remote: Enumerating objects: 462, done.
remote: Counting objects: 100% (462/462), done.
remote: Compressing objects: 100% (264/264), done.
remote: Total 8059 (delta 297), reused 316 (delta 198), pack-reused 7597
Receiving objects: 100% (8059/8059), 8.80 MiB | 22.53 MiB/s, done.
Resolving deltas: 100% (5839/5839), done.
     |████████████████████████████████| 3.0MB 3.5MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 194kB 30.9MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
Traceback (most recent call last):
  File "deep-person-reid/setup.py", line 47, in <module>
    version=find_version(),
  File "deep-person-reid/setup.py", line 16, in find_version
    with open(version_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'torchreid/__init__.py'


In [2]:
!python setup.py develop

/content/deep-person-reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '
Compiling torchreid/metrics/rank_cylib/rank_cy.pyx because it changed.
[1/1] Cythonizing torchreid/metrics/rank_cylib/rank_cy.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/deep-person-reid/torchreid/metrics/rank_cylib/rank_cy.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running develop
running egg_info
creating torchreid.egg-info
writing torchreid.egg-info/PKG-INFO
writing dependency_links to torchreid.egg-info/dependency_links.txt
writing requirements to torchreid.egg-info/requires.txt
writing top-level names to torchreid.egg-info/top_level.txt
writing manifest file 'torchreid.egg-info/SOURCES.txt'

In [0]:
import torchreid

In [4]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_crop']
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 5931 KB/s, 25 seconds passed
Extracting "/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    | 

In [5]:
model = torchreid.models.build_model(
    name='resnet50',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [0]:
train_loader=datamanager.train_loader

In [10]:
for imgs,pids,camids,_ in train_loader:
  print(camids)

tensor([4, 5, 4, 0, 2, 3, 5, 5, 3, 1, 5, 5, 4, 5, 5, 5, 4, 2, 4, 0, 5, 0, 3, 1,
        0, 2, 0, 3, 0, 4, 4, 1])
tensor([2, 1, 2, 4, 3, 4, 1, 4, 4, 5, 1, 3, 5, 5, 1, 4, 1, 5, 2, 4, 2, 0, 5, 0,
        5, 5, 0, 0, 1, 5, 1, 2])
tensor([5, 5, 3, 0, 1, 4, 0, 3, 4, 5, 0, 5, 0, 4, 5, 2, 4, 1, 4, 5, 1, 0, 4, 0,
        0, 0, 4, 2, 3, 5, 1, 0])
tensor([2, 4, 1, 1, 2, 3, 1, 2, 4, 4, 5, 2, 3, 5, 0, 1, 2, 5, 4, 2, 2, 5, 2, 2,
        2, 0, 2, 5, 5, 5, 2, 4])
tensor([5, 4, 2, 2, 2, 5, 5, 1, 1, 4, 5, 4, 5, 2, 5, 5, 4, 5, 4, 4, 4, 1, 2, 0,
        5, 2, 3, 0, 5, 2, 2, 2])
tensor([2, 0, 2, 4, 5, 0, 4, 5, 5, 1, 5, 0, 0, 0, 5, 5, 2, 4, 5, 1, 5, 1, 1, 2,
        1, 0, 5, 5, 1, 2, 5, 5])
tensor([5, 5, 5, 5, 2, 2, 3, 0, 0, 3, 3, 0, 5, 1, 0, 0, 1, 2, 5, 2, 2, 2, 2, 5,
        2, 1, 2, 1, 5, 2, 2, 2])
tensor([5, 0, 5, 2, 1, 4, 4, 0, 0, 4, 2, 5, 0, 1, 3, 4, 5, 0, 0, 4, 0, 5, 4, 4,
        1, 4, 2, 0, 1, 2, 0, 5])
tensor([4, 5, 2, 2, 5, 5, 1, 4, 0, 1, 1, 2, 5, 2, 4, 2, 5, 0, 1, 2, 5, 0, 4, 2,
        5, 0, 5,

KeyboardInterrupt: ignored